In [6]:
from secret_key import secret_key_1
import os
from langchain_openai import OpenAI

In [7]:
os.environ['OPENAI_API_KEY'] = secret_key_1

In [8]:
llm = OpenAI(temperature=0.6)
name = llm("Give me a sports trivia question about IPL and options for the same")
name

'\n\nWhat team holds the record for the most consecutive wins in a single IPL season?\n\nA) Chennai Super Kings\nB) Mumbai Indians\nC) Kolkata Knight Riders\nD) Sunrisers Hyderabad'

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

ipl_category = PromptTemplate(
    input_variables = ['category'],
    template = "Give me a sports trivia about Indian Premier League {category} with options. I'll give you an answer for the same."
)

category_chain = LLMChain(llm=llm, prompt = ipl_category, output_key = "Question")

ans_prompt = PromptTemplate(
    input_variables = ['option_selected'],
    template = "Is {option_selected} is the right answer?"
)

ans_chain = LLMChain(llm=llm, prompt = ans_prompt, output_key = "User_Answer")

In [13]:
category_chain.invoke('Batsman')

{'category': 'Batsman',
 'text': '\n\nQuestion: Who holds the record for the most centuries in the Indian Premier League?\n\nA) Virat Kohli\nB) Rohit Sharma\nC) Chris Gayle\nD) AB de Villiers\n\nAnswer: C) Chris Gayle'}

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


ipl_category = PromptTemplate(
    input_variables = ['category'],
    template = "Give me a sports trivia about Indian Premier League {category} with options. I'll give you an answer for the same."
)

category_chain = LLMChain(llm=llm, prompt = ipl_category, output_key = "category_selected")

ans_prompt = PromptTemplate(
    input_variables = ['category_selected'],
    template = "Is {category_selected} is the right answer?"
)

ans_chain = LLMChain(llm=llm, prompt = ans_prompt, output_key = "user_answer")

prompt_chain = SequentialChain(chains = [category_chain, ans_chain],
                              input_variables = ['category'],
                              output_variables = ['category_selected', "user_answer"]
)

prompt_chain.invoke({'category':'Bowler'})

{'category': 'Bowler',
 'category_selected': '\n\nQuestion: Who is the only Indian Premier League bowler to take a hat-trick in the final of the tournament?\n\nA) Lasith Malinga\nB) Amit Mishra\nC) Sunil Narine\nD) Rohit Sharma\n\nAnswer: B) Amit Mishra',
 'user_answer': ''}

{'category': 'Bowler', 'category_selected': '\n\nQuestion: Who is the only bowler in Indian Premier League history to take a hat-trick in the final match?\n\nA) Lasith Malinga\nB) Amit Mishra\nC) Jaydev Unadkat\nD) Pravin Tambe\n\nAnswer: A) Lasith Malinga'}


Enter your category_selected:  A


{'category': 'Bowler', 'category_selected': 'A', 'user_answer': '\n\nIt depends on the context of the question. Without more information, it is impossible to determine if A is the correct answer.'}
Final user answer: 

It depends on the context of the question. Without more information, it is impossible to determine if A is the correct answer.


In [26]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm = llm,
    memory = memory
)



In [30]:
convo.run("Give me a question about Indian Premier League Batsman with options. I'll give you an answer for the same.")


' How about this one: Who holds the record for the highest individual score in an IPL match? Your options are Chris Gayle, AB de Villiers, Rishabh Pant, and Shane Watson.'

In [31]:
convo.run("Chris Gayle")

' Correct! Chris Gayle holds the record for the highest individual score in an IPL match, scoring 175 runs off just 66 balls in 2013.'

In [32]:
convo.run("tell me more about the above match")

" Sure! The match was between the Royal Challengers Bangalore and the Pune Warriors India. Gayle's innings included 13 fours and 17 sixes, breaking numerous records and earning him the Man of the Match award. It was also the fastest century in T20 cricket at the time, achieved in just 30 balls. The match also saw a total of 263 runs scored, making it the highest scoring T20 match in history at the time."

In [33]:
convo.run("what is 10+19")

'  29'

In [35]:
convo.run("tell me more about the above cricket match")

" I'm sorry, I do not have enough context to accurately answer your question. Can you provide more information about the specific cricket match you are referring to?"